In [ ]:
from getStats import *
import requests
import pandas as pd
import time

In [ ]:
#Getting the API Keys from 
with open("keys.txt") as f:
  api = f.readlines()[0]

#Getting the list of igns provided for the program
with open("igns.txt") as g:
  igns = g.readlines()

In [ ]:
def getMatchStats(matchData, ign,api, df):
    #playerStats=[]
    kdas = 0
    visionScores = 0
    ccTime = 0
    topLane = 0
    csds = 0
    csdCount = 0
    wards = 0
    for match in matchData:
        url = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(match) +"?api_key="+ api
        response = requests.get(url).json()
        userIndex = getParticipantID(response, ign)
        kdas += calculateKDA(response, userIndex)
        visionScores += calculateVisionScore(response, userIndex)
        ccTime += calculateCCTime(response, userIndex)
        lane = getLaneStats(response, userIndex)
        wards += getWards(response, userIndex)
        if lane == 'TOP':
            topLane += 1
        #Riot's API will occasionally be unable to specify a lane opponent for a player, therefore omitting a CSD value as well. Only the matches with clearly defined CSDs are included in the average calculation.    
        csd = getCSD(response, userIndex, lane)
        if csd != False:
            csds += csd
            csdCount +=1
    try:
    	row = {'ign':ign, 'avgKDA':kdas/len(matchData), 'avgVisionScore':visionScores/len(matchData), 'avgWards':wards/len(matchData), 'percentTop':topLane/len(matchData), 'avgCSD':csds/csdCount}
    except:
    	row = {'ign':ign, 'avgKDA':kdas/len(matchData), 'avgVisionScore':visionScores/len(matchData), 'avgWards':wards/len(matchData), 'percentTop':topLane/len(matchData), 'avgCSD':0}
    print(row)
    
    return row

In [ ]:
def getAllMatchStats(matchData, ign,api, df):
    for match in matchData:
        url = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(match) +"?api_key="+ api
        response = requests.get(url).json()
        stats = pd.DataFrame(response['info']['participants'])

In [ ]:
#Building the dataset for the selected player stats
count = 0
df = pd.DataFrame()
	for ign in igns:
    #Wrapping in multiple try-excepts to account for rate limits and internet connectivity issues that may potentially arise
		try:
			accountID=getSummonerID(ign, api)
		except:
			print(ign)
			continue
		try:
			matchIDs = getMatchData(accountID, api)
		except:
			print(ign)
			continue
		try:
			row = getMatchStats(matchIDs, ign ,api, df)
		except:
			print(ign)
			continue
    
		df = df.append(row, ignore_index=True)
		df.to_csv("cleanedStats.csv")
		print(ign + "Row Added")
		time.sleep(3)
		count+=1
		if count == 5:
			time.sleep(100)
			print("Approaching Request limit, sleeping now")
			count=0
		df.to_csv("temp.csv")

In [ ]:
#Building the dataset for all of the player stats
count = 0
df = pd.DataFrame()
	for ign in igns:
    #Wrapping in multiple try-excepts to account for rate limits and internet connectivity issues that may potentially arise
		try:
			accountID=getSummonerID(ign, api)
		except:
			print(ign)
			continue
		try:
			matchIDs = getMatchData(accountID, api)
		except:
			print(ign)
			continue
		try:
			stats = getAllMatchStats((matchIDs, ign ,api, df)
		except:
			print(ign)
			continue
    df = df.append(stats, ignore_index=True)
		df.to_csv("allStats.csv")
		print(ign + "Row Added")
		time.sleep(3)
		count+=1
		if count == 5:
			time.sleep(100)
			print("Approaching Request limit, sleeping now")
			count=0
		df.to_csv("allStats.csv")
    